## Gerenciamento de memória
***

Quando você cria uma variável `x = 10` você ta armazenando o valor 10 em memória e apontando a variável x para esse local na memória onde ta armazenado esse valor.

Se você fizer `y = x`, ambos tanto `y` quanto `x` vão apontar para o mesmo local na memória onde ta armazenado a variável 10.

In [1]:
x = 10
print(id(x))

94608059940040


In [2]:
y = x
print(id(y))

94608059940040


In [3]:
print(id(x) == id(y))

True


No momento que vc fala que `x = x + 1` ou `x += 1` você agora ta tirando o vinculo do valor 10 que tava armazenado na memória e criando um novo espaço em memória com o valor 11.

In [4]:
x += 1
print(id(x))
print(id(x) == id(y))

94608059940072
False


Se eu criar uma nova variável `z = 10`, o python não vai alocar outro espaço de memória ele vai ver que já existe esse valor em memória e vai apontar a variavel `z` nesse local da memória.

In [5]:
z = 10
print(id(z))
print(id(z) == id(y))

94608059940040
True


Ou seja, Python inteligentemente reutiliza valores já alocados para novos valores, economizando espaço em memória. Mas o que acontece com os dados em memória quando não são mais referenciados por nenhum objeto? No momento que um valor em memória fica sem ninguém apontando para ele, o **Garbage Collector** remove ele da memória. O algorítmo utilizado pelo Garbage Collector do python é chamado de **Reference Counting**.

Diferentes linguagens de programaçao utilizam diferentes algorítmos para o Garbage Collector.

Ou seja:
* Métodos e variáveis são criadas na memória stack;
* Os objetos e instâncias são criadas na memória heap;
* Um novo stack é criado durante a invocação de uma função ou método;
* Stacks sçai destruidas sempre que uma função ou método retorna valor;
* Garbage Collector é um mecanismo para limpar Dead Objects na memória.

***
### GIL - Python Global Interpreter Lock
***

O python global interpreter lock, ou simplismente GIL, é um mutex (ou lock) que permite que apenas uma thread tome conta do interpretador python.

Isso significa que somente uma thread pode estar em um estado de execução em qualquer ponto do tempo.

O impacto do GIL não é comumente visível para desenvolvedores que executam programas single-thread, mas pode ser uma dor de cabeça para programas que precisam de tempo de resposta em códigos multi-thread.

Desde que o GIL permite apenas uma thread a ser executada, mesmo em computadores multi-threads com arquitetura que permite utilizar mais de um CPU ou core, o GIL tem ganhado reputação como recurso 'indecente' do python.

O python utiliza o algorítmo reference counting para gerenciamento de memória, isso significa que para cada objeto criado python mantém uma variável de contagem de referência que guarda quantas referências apontam para o objeto. Quando o contador de referência chega a zero, a memória ocupada é liberada.

In [6]:
import sys
a = []
b = a
print(sys.getrefcount(a))  # essa função tb ta referenciando a

3


In [7]:
b = None
print(sys.getrefcount(a))

2


O problema é que essa forma de gerenciamento de memória utilizando reference counting precisa de proteção para um fenômeno chamado `race conditions`, onde duas threads aumenta ou diminuem seu valor simultaneamente.

Se isso acontecer, poderá causar problemas de memória que nunca é liberada, ou ainda pior, liberação incorreta de memória enquanto ainda existe referência para o objeto.

E isso causa `crashs` ou outros bugs esquisitos no seu programa.

Este reference counting pode ser mantido seguro adicionando 'locks' em toda estrutura de dados que são compartilhadas via threads. Desta forma eles não são modificados de forma inconsistente.

O problema é que adicionar 'locks' em cada objeto ou grupo de objetos significa que múltiplos locks irão existir, e isso irá causar um outro problema - `Deadlocks` que é quando um lock não finaliza nunca (deadlocks só podem existir se existe mais de um lock). Outro efeito colateral seria decaimento da performace causada pela repetida aquisição e liberação de locks.

O GIL aplica na regra de execução de qualquer código python o single lock previnindo qualquer deadlock, o que por outro lado transforma qualquer código python em single-thread.

Vale mencionar que o GIL apesar de ser usado também em outras linguagens de programação como Ruby, não é a única solução.

Algumas linguagens evitam o uso do GIL para gerenciamento de memória em thread utilizando abordagens diferentes do reference couting que o python utiliza.

Por exemplo, uma das abordagens que outras linguagens utilizam é o compilador JIT - Just in Time, como o Java.

Vamos ver como isso impacta no código.

In [8]:
import time
contador = 50_000_000

In [9]:
def contagem_regressiva(n):
    while n > 0:
        n -= 1

In [10]:
# SINGLE THREAD (MAIN)
inicio = time.time()
contagem_regressiva(contador)
fim = time.time()
print(f"Tempo em segundos: {fim - inicio}")

Tempo em segundos: 4.774857759475708


In [11]:
# MULTI-THREADS
from threading import Thread
t1 = Thread(target=contagem_regressiva, args=(contador//2,))
t2 = Thread(target=contagem_regressiva, args=(contador//2,))
inicio = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
fim = time.time()
print(f"Tempo em segundos: {fim - inicio}")

Tempo em segundos: 4.7575788497924805


Mesmo utilizando multhread a diferença não é muito grande por causa do GIL, no final das contas vai ser 2 threads rodando como se fosse uma. A utilização do GIL prejudica a real utilização de multi-cores nas máquinas, o que torna os projetos python lentos em alguns casos. Por outro lado, o GIL torna as aplicações single-thread muito performaticas, e a grande maioria das aplicações não precisam utilizar multi-threads.

E como lidar com o GIL? Caso você tenha problemas com o GIL, você pode utilizar multi-processing ao invés de multithreading. Utilizando processos ao invés de threads cada processo python ganha seu próprio interpretador python e espaço em memória. Desta forma o GIL não será problema.

In [12]:
# MULTI-PROCESSING FORMA 01
from multiprocessing import Pool
pool = Pool(processes=2)
inicio = time.time()
p1 = pool.apply_async(contagem_regressiva, [contador//2])
p2 = pool.apply_async(contagem_regressiva, [contador//2])
inicio = time.time()
pool.close()
pool.join()
fim = time.time()
print(f"Tempo em segundos: {fim - inicio}")

Tempo em segundos: 2.6110236644744873


In [13]:
# MULTI-PROCESSING FORMA 02
from multiprocessing import Process
p1 = Process(target=contagem_regressiva, args=(contador//2,))
p2 = Process(target=contagem_regressiva, args=(contador//2,))
inicio = time.time()
p1.start()
p2.start()
p1.join()
p2.join()
fim = time.time()
print(f"Tempo em segundos: {fim - inicio}")

Tempo em segundos: 2.8882246017456055


Multi-processing são mais pesados que multi-threading, ou seja, demanda mais recurso da máquina e lembre-se que para cada processo, teremos um ambiente python próprio. Um interpretado que não tem GIL é o **[PyPy](https://pypy.org/)**, que é um uma implementação do python sem o GIL.